In [1]:
import sunpy.map
import os
import re
from datetime import datetime
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import fits
from matplotlib.colors import LogNorm, Normalize
import image_registration
from PIL import Image
from IPython.display import HTML
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm
import time
import itertools
import numpy as np
import cv2
%matplotlib notebook

In [2]:
file_path = "E:/python/projects/alfven/data/hrieuvopn/offset_data/10moving/"
files = [os.path.abspath(os.path.join(file_path, file)) for file in os.listdir(file_path) if file.endswith('.fits')]
def extract_datetime(file_name):
    match = re.search(r'\d{8}T\d{6}', file_name)
    if match:
        return datetime.strptime(match.group(), '%Y%m%dT%H%M%S')
    else:
        return datetime.min

files = sorted(files, key=extract_datetime)
file_fsi = "E:/python/projects/alfven/data/solo_L2_eui-fsi174-image_20220330T042045225_V02.fits"
file_fsi_L1 = "E:/python/projects/alfven/data/solo_L1_eui-fsi174-image_20220330T042045225_V02.fits"

定义一些绘图函数

In [3]:
# 绘制图像
def draw_hri_fig(hri_map, rotate=False, **kwargs):
    """
    绘制sunpy.map.Map对象图像
    :param hri_map: 文件名或sunpy.map.Map对象
    :param rotate: bool，True则按照CROTA对图像进行旋转
    :param kwargs: 传递给sunpy.map.Map.plot()方法的其他参数
    :return: none
    """
    if type(hri_map) == str:
        hri_map = sunpy.map.Map(hri_map)
    if rotate:
        hri_map = hri_map.rotate()
    fig = plt.figure()
    ax = fig.add_subplot(projection=hri_map)
    hri_map.plot(axes=ax, **kwargs)
    
# 绘制submap
def get_submap(hri_map, c_x, c_y, l, draw=True, **kwargs):
    """
    得到sunpy.map.Map对象图像的正方形submap
    :param hri_map: 文件名或sunpy.map.Map对象
    :param c_x: submap中心处的角秒横坐标
    :param c_y: submap中心处的角秒纵坐标
    :param l: 正方形图像的像素
    :param draw: bool，若为True则会绘制submap
    :param kwargs: 传递给sunpy.map.Map.plot()方法的其他参数
    :return: 原map的submap，sunpy.map.Map对象
    """
    if type(hri_map) == str:
        hri_map = sunpy.map.Map(hri_map)
    
    origin = SkyCoord(c_x*u.arcsec, c_y*u.arcsec, frame=hri_map.coordinate_frame)
    out_shape = (l, l)
    out_header = sunpy.map.make_fitswcs_header(
        out_shape,
        origin,
        scale=[0.492, 0.492]*u.arcsec/u.pix,
        projection_code="TAN"
    )
    supplement_list = [
        'DSUN_OBS',
        'RSUN_OBS',
        'RSUN_ARC',
        'FILENAME'
    ]
    unit_trasnform = {
        'CUNIT1': 'arcsec',
        'CUNIT2': 'arcsec',
        'CDELT1': 0.492,
        'CDELT2': 0.492,
        'CRVAL1': c_x,
        'CRVAL2': c_y
        }
    
    out_map = hri_map.reproject_to(out_header)
    for key in supplement_list:
        out_map.meta[key] = hri_map.meta[key]
    for key, value in unit_trasnform.items():
        out_map.meta[key] = value
        
    if draw:
        draw_hri_fig(out_map, **kwargs)
    
    return out_map

# 移动图像
def offset_map(hri_map, offset, draw=False, unit='pix', **kwargs):
    """
    对图像进行平移
    :param hri_map: 文件名或sunpy.map.Map对象
    :param offset: 横纵坐标的平移量
    :param draw: bool，True则绘制平移后的图像
    :param unit: pix或arcsec 
    :param kwargs: 传递给sunpy.map.Map.plot()方法的其他参数
    :return: 平移后的map，sunpy.map.Map对象
    """
    if type(hri_map) == str:
        hdu = fits.open(hri_map)
        header = hdu[-1].header
        data = hdu[-1].data
    else:
        header = hri_map.fits_header
        data = hri_map.data
        
    if unit == 'pix':
        cdelt = header['CDELT1']
        offset = [o*cdelt for o in offset]
    
    header['CRVAL1'] += offset[0]
    header['CRVAL2'] += offset[1]
    
    offset_hri_map = sunpy.map.Map(data, header)
    
    if draw:
        draw_hri_fig(offset_hri_map, **kwargs)
    
    return offset_hri_map

# 循环显示需要比较的两张图  
def compare_gif(fsi, hri, filename_gif, c_x=-100, c_y=-2800, l=2500, offset=None, unit='pix'):
    """
    将hri图像和fsi图像在一个动图中循环展示，进行肉眼比较
    :param fsi: fsi数据的文件名或sunpy.map.Map对象
    :param hri: hri数据的文件名或sunpy.map.Map对象
    :param filename_gif: 动图的保存位置
    :param c_x: submap中心处的角秒横坐标
    :param c_y: submap中心处的角秒纵坐标
    :param l: 正方形图像的像素
    :param offset: 将hri图像进行像素偏移的横纵坐标元组，默认为none
    :return: HTML对象，用于在notebook展示动图
    :param unit: pix或arcsec
    """
    if type(fsi) == str:
        fsi = get_submap(fsi, c_x=c_x, c_y=c_y, l=l, draw=False)

    if offset is not None:
        hri = offset_map(hri, offset, unit=unit)
        hri = get_submap(hri, c_x=c_x, c_y=c_y, l=l, draw=False)
    else:
        if type(hri) == str:
            hri = get_submap(hri, c_x=c_x, c_y=c_y, l=l, draw=False)
        
    with plt.ioff():
        fig = plt.figure()
        ax = plt.subplot(projection=fsi)
        fsi.plot(axes=ax, norm=Normalize(), cmap='sdoaia171')
        fig.savefig('./fig/fsi.png', dpi=300)
        ax.clear()

        ax = fig.add_subplot(projection=hri)
        hri.plot(axes=ax, norm=Normalize(), cmap='sdoaia171')
        fig.savefig('./fig/hri.png', dpi=300)
    
        image_files = ['./fig/fsi.png', './fig/hri.png']

        images = [Image.open(image) for image in image_files]

        images[0].save(filename_gif, save_all=True, append_images=images[1:], duration=500, loop=0)
        
    current_directory = os.getcwd()
    relative_path = os.path.relpath(filename_gif, current_directory)
    timestamp = int(time.time())
    
    return HTML(f'<img src={relative_path}?{timestamp} height="480">')

预览图像，找一个比较合适的图像范围进行比较

In [ ]:
draw_hri_fig(files[0], rotate=True)

取中心坐标(-100, -2700), 边长为1200像素（约600arcsec）的正方形

In [ ]:
_ = get_submap(file_fsi, c_x=-100, c_y=-2700, l=1200, norm=Normalize())

# 使用chi2_shift进行比较

In [6]:
data1 = get_submap(files[0], c_x=-100, c_y=-2700, l=1200, draw=False).data
data2 = get_submap(files[100], c_x=-100, c_y=-2700, l=1200, draw=False).data
# 归到0-255的范围
scale_data1 = (data1 - np.min(data1)) / (np.max(data1) - np.min(data1)) * 255
scale_data2 = (data2 - np.min(data2)) / (np.max(data2) - np.min(data2)) * 255

dx, dy = image_registration.chi2_shift(scale_data1, scale_data2, return_error=False, zeromean=True, max_auto_size=9600)

绘制强度分布图

In [ ]:
flat_data1 = data1.flatten()
flat_data2 = data2.flatten()
fig, ax = plt.subplots(figsize=(7, 4))
_ = ax.hist(flat_data1, bins='auto', density=True, alpha=0.5, color='blue', label='hri intensity')
_ = ax.hist(flat_data2, bins='auto', density=True, alpha=0.5, color='green', label='fsi intensity')
ax.grid()
ax.set_title('hri and fsi intensity probability')
ax.set_xlabel('intensity')
ax.set_ylabel('probability')
ax.legend(loc='upper right')
fig.savefig('./fig/probability.png', dpi=300)

归到0-255

In [ ]:
flat_scale_data1 = scale_data1.flatten()
flat_scale_data2 = scale_data2.flatten()
fig, ax = plt.subplots(figsize=(7, 4))
_ = ax.hist(flat_scale_data1, bins='auto', density=True, alpha=0.5, color='blue', label='hri scaled intensity')
_ = ax.hist(flat_scale_data2, bins='auto', density=True, alpha=0.5, color='green', label='fsi scaled intensity')
ax.grid()
ax.set_title('hri and fsi scaled intensity probability')
ax.set_xlabel('intensity')
ax.set_ylabel('probability')
ax.legend(loc='upper right')
fig.savefig('./fig/probability.png', dpi=300)

In [ ]:
compare_gif(file_fsi, files[0], 'E:/python/projects/alfven/fig/compare.gif', offset=(dx, dy), c_x=-200, c_y=-2800, l=1600)

In [ ]:
compare_gif(file_fsi, files[0], 'E:/python/projects/alfven/fig/compare_no_offset.gif', c_x=-200, c_y=-2800, l=1600)

定义一个函数，用于将hri图像转换到正确坐标下

In [ ]:
def offset_hri(hri_file, fsi_file, c_x=-200, c_y=-2600, l=800, savepath=None, return_map=False):
    hri_data = get_submap(hri_file, c_x=c_x, c_y=c_y, l=l, draw=False).data
    fsi_data = get_submap(fsi_file, c_x=c_x, c_y=c_y, l=l, draw=False).data
    offset_pix_x, offset_pix_y = image_registration.chi2_shift(hri_data, fsi_data, return_error=False)
    offset_hri_map = offset_map(hri_file, offset=(offset_pix_x, offset_pix_y))
    transformed_hri_map = get_submap(offset_hri_map, c_x=-100, c_y=-2800, l=2500, draw=False)
    filename = os.path.join(savepath, transformed_hri_map.fits_header['FILENAME'])
    if savepath is not None:
        transformed_hri_map.save(filename, filetype='fits', overwrite=True)
    if return_map:
        return transformed_hri_map

进行平移，得到矫正过后的图像

In [ ]:
# save_path = "E:/python/projects/alfven/data/transform_data/"
# 
# with ThreadPoolExecutor() as executor:
#     futures = {executor.submit(offset_hri, file, file_fsi, save_path): file for file in files}
# 
#     for future in tqdm(as_completed(futures), total=len(files), desc="Processing files"):
#         future.result()

# 使用workshop中的方法进行矫正

In [12]:
def get_diff(hri, fsi, c_x, c_y, l, offset):
    hri_offset = offset_map(hri, offset, unit='arcsec')
    hri_data = get_submap(hri_offset, c_x=c_x, c_y=c_y, l=l, draw=False).data
    fsi_data = get_submap(fsi, c_x=c_x, c_y=c_y, l=l, draw=False).data
    
    def normalize(image):
        return (image - np.mean(image)) / np.std(image)
    
    return np.nansum(np.abs(normalize(hri_data)-normalize(fsi_data)))

In [13]:
def iterate_alignment(hri, fsi, c_x, c_y, l, accuracy, offset_range):
    accuracy = np.array(accuracy)
    range_x, range_y = offset_range[0], offset_range[1]
    min_diff = np.inf
    best_offset = (0, 0)
    for a in accuracy:
        offset_vect_x = np.arange(range_x[0], range_x[1]+a, a)
        offset_vect_y = np.arange(range_y[0], range_y[1]+a, a)
        offsets = list(itertools.product(offset_vect_x, offset_vect_y))
        
        with ThreadPoolExecutor() as executor:
            futures = {executor.submit(get_diff, hri, fsi, c_x, c_y, l, offset): offset for offset in offsets}
            for future in tqdm(as_completed(futures), total=len(offsets), desc=f"computing best offset, accracy: {a}", leave=False):
                diff = future.result()
                if diff < min_diff:
                    min_diff = diff
                    best_offset = futures[future]
                    
        range_x = (best_offset[0]-a, best_offset[0]+a)
        range_y = (best_offset[1]-a, best_offset[1]+a)
    
    return best_offset

先来手动调整，找一个合适的offset范围，可以看到，在角秒偏移为offset_crval1=20, offset_crval2=10时，比较吻合图像

In [ ]:
compare_gif(file_fsi, files[195], './fig/fsi_vs_hri.gif', l=1600, offset=(15, 8), unit='arcsec')

因此设定初始查找范围为offset_x_range=(17, 23), offset_y_range=(7, 13)

In [ ]:
best_x, best_y = iterate_alignment(files[196], files[195], c_x=-100, c_y=-2700, l=1200, 
                                   offset_range=((-3, 3), (-3, 3)), accuracy=[1, 0.5, 0.2, 0.05])

In [ ]:
compare_gif(file_fsi, files[196], './fig/fsi_vs_hri.gif', l=1600, offset=(14, 8), unit='arcsec')

In [ ]:
file0 = files[295]
needed_files = files[295:504]

In [ ]:
offset_vect = []
for file in tqdm(needed_files):
    best_offset = iterate_alignment(file, file0, c_x=-100, c_y=-2700, l=1200, 
                                   offset_range=((-3, 3), (-3, 3)), accuracy=[1, 0.5, 0.2, 0.05])
    offset_vect.append(best_offset)

在工作站得到平移数组，查看效果

In [7]:
offset_vect = np.load('./data/offset_array.npy')
offset_vect = np.round(offset_vect, 2)
st_files = files[295:504]

In [14]:
x0, y0 = iterate_alignment(files[295], file_fsi, c_x=-100, c_y=-2700, l=1200, 
                           offset_range=((16, 19), (6, 9)), accuracy=[1, 0.5, 0.2, 0.05])
x0 = np.round(x0, 2)
y0 = np.round(y0, 2)

computing best offset, accracy: 1.0:   0%|          | 0/16 [00:00<?, ?it/s]

computing best offset, accracy: 0.5:   0%|          | 0/25 [00:00<?, ?it/s]

computing best offset, accracy: 0.2:   0%|          | 0/42 [00:00<?, ?it/s]

computing best offset, accracy: 0.05:   0%|          | 0/90 [00:00<?, ?it/s]

In [22]:
compare_gif(file_fsi, files[295], './fig/fsi_vs_hri.gif', l=1600, offset=(x0, y0), unit='arcsec')

In [15]:
offset_file_fold = "E:/python/projects/alfven/data/hrieuvopn/offset_data/"

for i, file in tqdm(enumerate(st_files), total=len(st_files), desc='offseting'):
    offset = offset_vect[i]
    offseted_map = offset_map(file, offset=(x0, y0), unit='arcsec')
    offseted_map.save(offset_file_fold+offseted_map.fits_header['filename'], filetype='fits', overwrite=True)

offseting:   0%|          | 0/209 [00:00<?, ?it/s]

In [4]:
offset_file_path = "./data/hrieuvopn/offset_data/10moving/"
offset_files = [os.path.abspath(os.path.join(offset_file_path, file)) for file in os.listdir(offset_file_path) if file.endswith('.fits')]
def extract_datetime(file_name):
    match = re.search(r'\d{8}T\d{6}', file_name)
    if match:
        return datetime.strptime(match.group(), '%Y%m%dT%H%M%S')
    else:
        return datetime.min

offset_files = sorted(offset_files, key=extract_datetime)

In [5]:
compare_gif(offset_files[30], offset_files[31], './fig/fsi_vs_hri.gif',
            c_x=0, c_y=-3000, l=200, offset=None, unit='arcsec')